In [75]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from pandas import MultiIndex, Int64Index
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import optuna
import pandas as pd
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag



/var/folders/32/zv3nm8_12xb9ytd7vk7nnr6w0000gn/T/ipykernel_87566/4280354733.py:8: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [76]:
!python --version

Python 3.9.7


In [77]:
# !pip install optuna
# !pip install scipy
# !pip install autogluon
# !pip install xgboost
# !python3.9 -m pip install --upgrade pip
# !pip install bokeh==2.0.1


In [78]:
df_train = pd.read_csv('../data/train_encoded_pca_cum.csv')
df_test = pd.read_csv('../data/test_encoded_pca_cum.csv')
#train_encoded_pca_cum
#train_encoded_pca
#ex = pd.read_csv('../data/example-submission.csv')

In [79]:
df_test.head()

,size_sqft,property_type_cluster house,property_type_condo,property_type_corner terrace,property_type_executive condo,property_type_hdb,property_type_semi-detached house,property_type_terraced house,property_type_infrequent_sklearn,furnishing_partial,...,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
0,463,0,1,0,0,0,0,0,0,0,...,0.034716,-0.942867,-0.870046,0.544891,0.442697,1.227460,-0.261023,-0.960449,0.191969,-0.113819
1,1033,0,1,0,0,0,0,0,0,0,...,0.369292,-1.030715,-1.349437,-1.661337,0.821686,-0.620200,0.470905,0.296070,0.190211,-0.479004
2,570,0,1,0,0,0,0,0,0,0,...,0.457908,1.409650,0.053265,-0.590787,-0.330942,0.744301,0.286882,0.111819,-0.054269,0.449271
3,1216,0,0,0,0,1,0,0,0,0,...,-0.276300,-0.634599,-0.234421,0.553224,-1.217748,-1.105000,2.080035,-1.147152,-0.277603,0.817754
4,936,0,0,0,0,1,0,0,0,0,...,1.797542,1.959214,-1.115832,0.824612,-0.043025,1.433574,-0.593093,-1.003330,0.771620,0.757636


In [80]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df_train, test_size=0.2)

In [81]:
X_train = train.drop(['price'], axis=1)

X_valid = valid.drop(['price'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
#df_test= df_test.drop(['size_sqft'], axis=1)

np.random.seed(0)
idxs = np.arange(X_train.shape[0])
np.random.shuffle(idxs)
df_test.columns.difference(X_train.columns)


Index([], dtype='object')

In [82]:
X_train.columns.difference(df_test.columns)
df_test.head()

,size_sqft,property_type_cluster house,property_type_condo,property_type_corner terrace,property_type_executive condo,property_type_hdb,property_type_semi-detached house,property_type_terraced house,property_type_infrequent_sklearn,furnishing_partial,...,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11
0,463,0,1,0,0,0,0,0,0,0,...,0.034716,-0.942867,-0.870046,0.544891,0.442697,1.227460,-0.261023,-0.960449,0.191969,-0.113819
1,1033,0,1,0,0,0,0,0,0,0,...,0.369292,-1.030715,-1.349437,-1.661337,0.821686,-0.620200,0.470905,0.296070,0.190211,-0.479004
2,570,0,1,0,0,0,0,0,0,0,...,0.457908,1.409650,0.053265,-0.590787,-0.330942,0.744301,0.286882,0.111819,-0.054269,0.449271
3,1216,0,0,0,0,1,0,0,0,0,...,-0.276300,-0.634599,-0.234421,0.553224,-1.217748,-1.105000,2.080035,-1.147152,-0.277603,0.817754
4,936,0,0,0,0,1,0,0,0,0,...,1.797542,1.959214,-1.115832,0.824612,-0.043025,1.433574,-0.593093,-1.003330,0.771620,0.757636


In [83]:
#scaler = preprocessing.MinMaxScaler().fit(X_train)

# xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, eval_metric=["error"])
# xgb_model.fit(X_train, y_train)

# # Predict labels for test data
# yt_pred = xgb_model.predict(X_train)
# yv_pred = xgb_model.predict(X_valid)

# print('Training RMSE: {:.3f}'.format(mean_squared_error(yt_pred, y_train, squared=False)))
# print('Test RMSE: {:.3f}'.format(mean_squared_error(yv_pred, y_valid, squared=False)))

# print(f'Percentage error for training data {((abs(yt_pred - y_train)/y_train)*100).mean()}')
# print(f'Percentage error for validation data {((abs(yv_pred - y_valid)/y_valid)*100).mean()}')

# xgb.plot_importance(xgb_model, height = 5, max_num_features=20)

# score = cross_val_score(xgb_model, X_train, y_train, n_jobs=-1, cv=5, scoring="r2")
# r2_mean = score.mean()

# print(f"R2 mean: {r2_mean}")
#yTest = xgb_model.predict(df_test)

# yTest = yTest * df_test_size_sqft
# yTest = pd.DataFrame(yTest.values, columns=['Predicted'], index=df_test.index)

# yTest.to_csv('../predictions/xgboost.csv')


Tabular Predictor

In [84]:
metric_reg = 'mean_absolute_error'
label_reg = 'price_per_sqft'
train_data_reg = TabularDataset(X_train) 
valid_data_reg = TabularDataset(X_valid)
test_data_reg = TabularDataset(df_test)
#train_data_reg = train_data_reg.drop(['price', 'size_sqft'], axis=1)
test_data_reg = test_data_reg.drop(['size_sqft'], axis=1)

predictor = TabularPredictor(label = label_reg, eval_metric = metric_reg)

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {
 'GBM': gbm_options,
 'XGB': [{'booster':'gblinear','alpha':1, 'lambda':1 },
    {'booster':'gbtree','alpha':0.1, 'lambda':1 },
    {'booster':'gblinear','alpha':0.5, 'lambda':1 },
    {'booster':'gblinear','alpha':0.7, 'lambda':1 },
    {'booster':'gbtree','alpha':0.1, 'lambda':1 },
    {'booster':'gbtree','alpha':0.5, 'lambda':1 },
    {'alpha':1}],
'RF': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_types': ['regression', 'quantile']}}],
 'XT': [{'criterion': 'gini',
   'ag_args': {'name_suffix': 'Gini',
    'problem_types': ['binary', 'multiclass']}},
  {'criterion': 'entropy',
   'ag_args': {'name_suffix': 'Entr',
    'problem_types': ['regression']}},
  {'criterion': 'squared_error',
   'ag_args': {'name_suffix': 'MSE',
    'problem_types': ['regression', 'quantile']}}]
}

time_limit = 2*60  # train various models for ~2 min
num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



excluded_model_types = ['NN_TORCH', 'FASTAI', 'KNN', 'CAT', 'RF']
predictor_reg = predictor.fit(train_data_reg, excluded_model_types = excluded_model_types, time_limit = 900
,hyperparameters=hyperparameters
,tuning_data=valid_data_reg, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs
)


No path specified. Models will be saved in: "AutogluonModels/ag-20221019_103622/"
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20221019_103622/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    13773
Train Data Columns: 64
Tuning Data Rows:    3444
Tuning Data Columns: 64
Label Column: price_per_sqft
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (44625.0, 2.032620320855615, 1828.14799, 1223.50946)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
/usr/local/lib/python3.9/site-packages/autogluon/tabular/learner/default_learner.py:151: FutureWarning: iteritems is deprecated and wil

TypeError: string indices must be integers

In [ ]:
#/Users/nishita/Documents/Semester 1/CS5228/project/notebooks/AutogluonModels/ag-20221019_034135
folderName = "AutogluonModels/ag-20221019_034135/"
predictor = TabularPredictor.load(folderName)

predictor.fit_hyperparameters_
##test_data_reg = TabularDataset(df_test)
#test_data_reg

In [ ]:
folderName = "AutogluonModels/ag-20221019_103622/"
#With target encoding and less one hot encoding.
predictor = TabularPredictor.load(folderName)
results = predictor.fit_summary(show_plot=True)

def writePredictionsToFile(modelName, folder = folderName, refit = False):
    if(refit == True):
        modelName = modelName +  '_FULL'    
    y_pred = predictor.predict(test_data_reg, model=modelName)
    yTest = y_pred * df_test_size_sqft
    yTest = pd.DataFrame(yTest, columns=['Predicted'], index=df_test.index)
    yTest.to_csv(f'{folder}/predictions/{modelName}.csv')


In [ ]:
writePredictionsToFile('WeightedEnsemble_L2')
writePredictionsToFile('RandomForestMSE')
writePredictionsToFile('LightGBMLarge')
#writePredictionsToFile('CatBoost')
writePredictionsToFile('XGBoost')
writePredictionsToFile('ExtraTreesMSE')
writePredictionsToFile('LightGBM')
writePredictionsToFile('LightGBMXT')



In [ ]:
predictor.refit_full()